# Embedding Berufe to 300d Glove vocab, then clusetring
### https://deepset.ai/german-word-embeddings

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df_berufe = pd.read_csv('./data/df_mdb.csv')

In [3]:
df_berufe.head()

,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,...,12,13,14,15,16,1,17,18,19,BERUF_MAPPED
0,11000001,Abelein,Manfred,20.10.1930,Stuttgart,UNBEKANNT,17.01.2008,männlich,keine Angaben,katholisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jurist*in
1,11000002,Achenbach,Ernst,09.04.1909,Siegen,UNBEKANNT,02.12.1991,männlich,"verheiratet, 3 Kinder",evangelisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jurist*in
2,11000003,Ackermann,Annemarie,26.05.1913,Parabutsch,Jugoslawien,18.02.1994,weiblich,"verheiratet, 5 Kinder",katholisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hilfsreferentin
3,11000004,Ackermann,Else,06.11.1933,Berlin,UNBEKANNT,14.09.2019,weiblich,ledig,evangelisch,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mediziner/ Ärztin
4,11000005,Adam,Ulrich,09.06.1950,"Teterow, Kr. Teterow, Bezirk Neubrandenburg",UNBEKANNT,UNBEKANNT,männlich,"verheiratet, 2 Kinder",evangelisch,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,Naturwissenschaftler*in


In [5]:
df_berufe[['ID','BERUF']].groupby('BERUF').count().sort_values(by='ID', ascending=False).head()

,ID
BERUF,
Rechtsanwalt,182
Geschäftsführer,80
Landwirt,79
Angestellter,63
Rechtsanwalt und Notar,47


In [6]:
len(set(df_berufe.BERUF))

2201

In [22]:
from importlib import reload

import berufe_mapping 
reload(berufe_mapping)
from berufe_mapping import basic_cleaning_berufe

In [23]:
df_berufe.BERUF_MAPPED = df_berufe.BERUF.copy()
df_berufe = basic_cleaning_berufe(df_berufe, 'BERUF_MAPPED')
df_berufe[['ID','BERUF_MAPPED']].groupby('BERUF_MAPPED').count().sort_values(by='ID', ascending=False).head()

,ID
BERUF_MAPPED,
rechtsanwalt,373
landwirt,130
geschäftsführer,103
kaufmann,96
ingenieur,88


In [24]:
# already reduction: 2.2k => 0.9
len(set(df_berufe.BERUF_MAPPED))

1150

In [25]:
NUM_FREQ_JOBS = 256
frequent_jobs = df_berufe[['ID','BERUF_MAPPED']].groupby('BERUF_MAPPED').count().sort_values(by='ID', ascending=False).head(NUM_FREQ_JOBS).index
frequent_jobs_list = [job for job in frequent_jobs.str.lower()]
list(frequent_jobs_list)[:8]

['rechtsanwalt',
 'landwirt',
 'geschäftsführer',
 'kaufmann',
 'ingenieur',
 'volkswirt',
 'angestellter',
 'jurist']

In [26]:
# embedding source https://deepset.ai/german-word-embeddings
glove_path = os.path.join('data', 'GloVe_vectors.txt')

In [27]:
embeddings_index = {}
with open(glove_path) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 1309281 word vectors.


In [28]:
job_dict = {}
for job in frequent_jobs_list:
    if job in embeddings_index.keys():
        job_dict[job] = embeddings_index[job]
    else:
        print(f'sorry not found: {job}')

sorry not found: gewerkschaftssekretär
sorry not found: industriekaufmann
sorry not found: mitglied der volkskammer
sorry not found: politikwissenschaftler
sorry not found: landwirtschaftsmeister
sorry not found: kaufmännischer angestellter
sorry not found: vizepräsident dbt
sorry not found: universitätsprofessor
sorry not found: regierungsdirektor
sorry not found: vizepräsidentin dbt
sorry not found: verwaltungsangestellter
sorry not found: oberregierungsrat
sorry not found: ministerpräsident
sorry not found: oberstudiendirektor
sorry not found: ingenieur (fh)
sorry not found: geschäftsführerin
sorry not found: oberbürgermeister
sorry not found: gewerkschaftssekretärin
sorry not found: unternehmensberater
sorry not found: sozialwissenschaftlerin
sorry not found: versicherungskaufmann
sorry not found: sozialwissenschaftler
sorry not found: technischer angestellter
sorry not found: hauptgeschäftsführer
sorry not found: industriekauffrau
sorry not found: maschinenschlosser
sorry not foun

In [29]:
len(job_dict)

166

In [30]:
df_job_dict = pd.DataFrame(job_dict).T
df_job_dict

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
rechtsanwalt,-0.130809,0.465649,-0.019781,0.250159,0.166545,0.933898,0.143043,-0.006213,-0.543607,-0.291613,...,-0.384041,-0.600597,-0.075080,0.148264,-0.231038,-0.147893,0.029758,-0.144397,0.024036,-0.348432
landwirt,0.216409,0.387386,-0.001010,0.509018,-0.061581,0.585741,0.523000,-0.355446,-0.509728,0.081854,...,0.127742,0.194123,-0.044218,0.052170,-0.499352,0.166933,-0.714272,0.313118,0.183399,-0.685334
geschäftsführer,-0.004873,0.342043,0.629353,-0.128318,-0.059298,0.123296,0.492444,-0.112399,0.130681,-0.117017,...,0.273969,-0.063877,-0.351897,-0.166674,-0.131167,0.230641,-0.235241,-0.423941,-0.026832,-0.201920
kaufmann,0.120680,0.100275,-0.264444,0.677685,0.094593,0.657143,0.242560,0.116608,0.262535,-0.210317,...,0.474511,-0.027455,0.245017,-0.066342,-1.044692,-0.025385,-0.021669,-0.324546,-0.178431,-0.046742
ingenieur,0.159797,0.521172,-0.007730,-0.032616,0.200749,0.652369,-0.013746,0.150849,0.052613,-0.005535,...,-0.198476,-0.297136,-0.159677,-0.031706,-0.346212,0.164424,-0.022718,-0.046887,-0.051185,-0.564675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
philosophin,-0.779789,0.589368,-0.111773,0.553429,-0.595593,-0.327220,-0.263873,-0.485640,0.465220,-0.188167,...,0.103893,0.164703,0.513938,-0.324499,-0.407483,-0.570188,0.208172,-0.584982,-0.129306,0.155841
stadtrechtsrat,0.288723,0.085641,-0.108765,0.001381,0.158834,-0.234049,0.236337,-0.090364,0.127875,0.232049,...,0.113805,0.055612,-0.074416,-0.224184,0.139565,-0.531125,-0.065960,0.304350,0.172308,-0.149787
müllermeister,0.692379,0.098258,0.088115,0.396000,-0.441065,-0.180311,0.039852,-0.041023,-0.320001,-0.402433,...,-0.068104,0.741789,-0.117611,0.246355,-0.163446,0.049273,-0.199906,-0.157635,0.232808,0.093295
staatsanwalt,-0.703067,-0.368462,-0.512241,-0.483381,0.398008,0.510417,-0.493400,-0.470893,0.088544,-0.082400,...,-0.399301,-0.205852,-0.012993,-0.192548,0.068216,-0.138866,-0.223459,-0.102509,0.106927,-0.266591


In [31]:
from sklearn.cluster import KMeans, DBSCAN

NCLUSTER = 32
kmeans = KMeans(n_clusters=NCLUSTER).fit(df_job_dict)
#clustering = DBSCAN(eps=3, min_samples=2).fit(df_job_dict)


In [32]:
labels = kmeans.labels_
len(labels)

166

In [33]:
for cluster in range(NCLUSTER):
    print()
    print(f'cluster number {cluster}')
    print(df_job_dict.iloc[labels==cluster].index)


cluster number 0
Index(['redakteur', 'journalistin', 'redakteurin', 'referentin'], dtype='object')

cluster number 1
Index(['juristin', 'pädagogin', 'soziologin', 'politologin', 'historikerin',
       'chemikerin', 'psychologin', 'professorin', 'unternehmerin', 'biologin',
       'richterin', 'mathematikerin', 'publizistin', 'ökonomin',
       'philosophin'],
      dtype='object')

cluster number 2
Index(['verleger', 'buchhändler', 'buchdrucker'], dtype='object')

cluster number 3
Index(['chefredakteur', 'verlagskaufmann', 'verlagsleiter'], dtype='object')

cluster number 4
Index(['journalist', 'politologe', 'ökonom', 'schriftsteller', 'historiker',
       'soziologe', 'theologe', 'hochschullehrer', 'publizist', 'pädagoge',
       'biologe', 'psychologe', 'philologe'],
      dtype='object')

cluster number 5
Index(['volljurist', 'polizeibeamter', 'realschullehrer', 'elektromeister',
       'stadtamtmann', 'oberschulrat', 'institutsleiter', 'stadtkämmerer',
       'stadtdirektor', 'kri

In [34]:
df_job_dict.iloc[labels==6].index

Index(['lehrer', 'oberstudienrat', 'pfarrer', 'studienrat', 'rektor',
       'studiendirektor', 'oberlehrer', 'schulleiter', 'gymnasiallehrer'],
      dtype='object')